In [ ]:
import numpy as np
import scipy.stats as stats
from hansolo import *

Nb = 100
expert = "PWA"  # "EWA" or "PWA" or "Multilin"
learning = "random" # "blocs": objects presented by random blocs, or "random": each object is randomly chosen with replacement


M = 2502
M_obj = 500 #size of testing set
n_epochs = int(M/9) #number of epochs
dt = 0.002
N = 1000
T = N * dt

freq = 100  # firing rate input neurons +
freq2 = 150

alpha = np.array([100 * dt, 0])  # spontaneous rates of output neurons

p = freq * dt  # spiking proba input neurons
p2 = freq2 * dt

K_input = 12  # nb experts
n_input = 6  # nb input neurons
K_output = 2  # nb output neurons


obj = [
    [1, 0, 0, 1, 0, 0],
    [1, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 1],
    [0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0, 1],
]
# blue circle blue square blue triangle red circle red square red triangle gray circle gray square gray triangle



answers = np.zeros((Nb, n_epochs, M_obj))  # 0 if wrong, 1 if correct

W_output = np.load(f'/Users/sophiejaffard/Desktop/saves/{expert}_{learning}_W_HAN.npy')
W_output = W_output[:,:,9*np.arange(int(M/9)),:] #weights after each epoch



for nb in range(Nb):
    #creation of the list of objects (testing set)
    L=list_objects_random(obj,M_obj)

    F_input = np.zeros((n_input, n_epochs, M_obj))
    F_output=np.zeros((K_output, n_epochs, M_obj))

    
    for m_obj in range(M_obj):

        obj_cur = L[m_obj]
        cur_act = np.where(obj_cur == 1)[0]
        input_neurons=np.zeros((n_input, n_epochs, N))
        input_neurons[cur_act, :,:] = Poisson(freq, T, (cur_act.size, n_epochs, N))

        output_neurons=np.zeros((K_output, n_epochs, N))
        for m in range(n_epochs):

            # simulation of the output neurons
            
            for j in range(K_output):
                probas = alpha[j] + np.sum(
                    (
                        W_output[j, :n_input, m, nb][:, np.newaxis]
                        - W_output[j, n_input:, m, nb][:, np.newaxis]
                    )
                    * input_neurons[:,m,:],
                    axis=0,
                )
                clipped_probas = np.clip(probas, 0, 1)
                output_neurons[j, m, :] = stats.bernoulli.rvs(clipped_probas)

        # firing rates
        F_input[:, :, m_obj] = np.sum(input_neurons, axis=2) / T
        F_output[:, :, m_obj] = np.sum(output_neurons, axis=2) / T

        for m in range(n_epochs):
            if (in_A(obj_cur) and F_output[0, m, m_obj] > F_output[1, m, m_obj]) or (
                in_B(obj_cur) and F_output[1, m, m_obj] > F_output[0, m, m_obj]
            ):
                answers[nb, m, m_obj] = 1
        if m_obj%100==0:
            print(nb,m_obj)


np.save(f"{expert}_{learning}_HAN_test",answers)
#np.save(f"{expert}_{learning}_W_HAN", W_output)
#np.save(f"{expert}100F_in", F_output)
